In [42]:
from bs4 import BeautifulSoup as bs
from decimal import *
import sys
import os

time = {
    'measure':None,
    'longa':Decimal(4),
    'breve':Decimal(2),
    'whole':Decimal(1),
    'half':Decimal(1)/2,
    'quarter':Decimal(1)/4,
    'eighth':Decimal(1)/8,
    '16th':Decimal(1)/16,
    '32nd':Decimal(1)/32,
    '64th':Decimal(1)/64,
    '128th':Decimal(1)/128
}

guitar = {
    '24':'AG_nylon',
    '25':'AG_steel',
    '26':'EG_jazz',
    '27':'EG_clean',
    '28':'EG_muted',
    '29':'EG_overdrive',
    '30':'EG_distortion',
    '31':'EG_harmonics',
    '84':'EG_synth',
}

def ifDots(tag):
    if not tag.find('dots'):
        multi = 1
    else:
        dots = tag.find('dots').text
        if dots == '1':
            multi = 1.5
        elif dots == '2':
            multi = 1.75
        elif dots == '3':
            multi = 1.875
    return Decimal(multi)

In [43]:
def main(dirPath):
    for filename in os.listdir(dirPath):
        print filename
        with open(dirPath+filename, 'r') as f:
            mscx = bs(f.read(), 'xml')
        if mscx.find('Division').text != '480':
            print 'Continue at measure #' + measure['number'] + 'while the Division != 480'
            continue
        deterGuitarStaffs(mscx, filename)

In [47]:
#Determine which staffs play guitar
def deterGuitarStaffs(mscx, filename):
    for part in mscx.find_all('Part'):
        staffID = []
        #Labeled instrument
        labInst = ''
        #Represented instrument
        repInst = ''
        #Source instrument
        srcInst = ''
        for tag in part.find_all(['Staff', 'longName', 'instrumentId', 'program']):
            if tag.name == 'Staff':
                staffID.append(tag['id'])
            elif tag.name == 'longName':
                labInst = tag.text
            elif tag.name == 'instrumentId':
                repInst = tag.text
            else: #tag.name == 'program'
                srcInst = tag['value']
                break #while reaching 1st <program> in <Channel>
        
        #Pick guitar staffs for details extraction
        if srcInst in guitar and ('guitar' in repInst.lower() or 'guitar' in labInst.lower()):
            for ID in staffID:
                staff = mscx.select_one('Score > Staff:nth-of-type('+ID+')')
                print '[=============== Staff #' + staff['id'] + ' ===============]'
                beatsPerMeasure(staff, guitar[srcInst], filename)

In [48]:
#Extract beats in every track of a measure
#All break statements are to continue the outer loop "for measure" from inner loop "for tag"
#Statement continue in the ELSE block of inner loop "for tag" affects the outer loop "for measure"
def beatsPerMeasure(staff, srcGuitar, filename):
    for measure in staff.find_all('Measure'):
        track_0_time_acc = 0
        isTrack = False
        track_key = 'track0'
        time_acc = 0
        lastAvg = None
        thisMeasure = {track_key:''}
        for track in measure.find_all('track'):
            if thisMeasure.get('track'+track.text) is None:
                thisMeasure.update({'track'+track.text:''})
        track = 'single_track' if len(thisMeasure) == 1 else 'multi_track'
        if measure.find('TimeSig'):
            sigN = Decimal(measure.find('sigN').text)
            sigD = Decimal(measure.find('sigD').text)
            time['measure'] = sigN / sigD
            measureDivision = 480 * sigN
        if measure.get('len'):
            continue
        for tag in measure.find_all(['Rest','Chord','Tuplet','tick'], recursive=False):
            if tag.name == 'Chord' or tag.name == 'Rest':
                noteSign = '1' if tag.name == 'Chord' else '0'
                if track == 'multi_track' and noteSign == '1' and len(tag.find_all('Note', recursive=False)) > 1:
#                     print 'Continue at measure #' + measure['number'] + ' with more than 1 track and with a moment with double note or more'
                    break
                
                #track number changes from another to 0
                if not tag.find('track') and track_key != 'track0':
                    isTrack = False
                    track_key = 'track0'
                    if time_acc != time['measure']:
#                         print 'Continue at measure #' + measure['number'] + ', track number changes from another to 0'
                        break
                    time_acc = 0
                    lastAvg = None
                    
                #track number changes from 0 to another
                elif tag.find('track') and track_key == 'track0':
                    isTrack = True
                    track_key = 'track' + tag.find('track').text
                    lastAvg = None
                    
                #track number changes from nonzero to another nonzero
                elif tag.find('track') and track_key != 'track'+tag.find('track').text:
                    track_key = 'track' + tag.find('track').text
                    if time_acc != time['measure']:
#                         print 'Continue at measure #' + measure['number'] + ', track number changes from nonzero to another nonzero'
                        break
                    time_acc = 0
                    lastAvg = None
                    
                multi = ifDots(tag)
                if tag.find('Tuplet') and tag.find('Tuplet').text == tupletID and not tag.find('dots'):
                    multi *= tupletRatio
                elif tag.find('Tuplet') and tag.find('Tuplet').text == tupletID and tag.find('dots'):
#                     print 'Continue at measure #' + measure['number'] + ' with a dot found in tuplet'
                    break
                duTime = Decimal(time[tag.select('durationType')[0].text]) * multi
                
                #Record the length of this note, if it's a note or rest and if it's ascending or descending
                if noteSign == '1':
                    thisAvg = sum([float(note.find('pitch').text) for note in tag.find_all('Note', recursive=False)]) / len(tag.find_all('Note', recursive=False))
                    if lastAvg is not None:
                        if thisAvg > lastAvg:
                            ascendingSign = '1'
                        elif thisAvg < lastAvg:
                            ascendingSign = '-1'
                        else:
                            ascendingSign = '0'
                        thisMeasure[track_key] += str(duTime) + ',' + noteSign + ',' + ascendingSign + ';'
                    else:
                        thisMeasure[track_key] += str(duTime) + ',' + noteSign + ';'
                    lastAvg = thisAvg
                else:
                    thisMeasure[track_key] += str(duTime) + ',' + noteSign + ';'
                
                #To accumulate note length for special cases exclusion
                if isTrack == False:
                    track_0_time_acc += duTime
                else:
                    time_acc += duTime
                    if ticking is not None:
                        if ticking % measureDivision == 0:
                            ticking = None
                        else:
#                             print 'Continue at measure #' + measure['number'] + ', ticking % measureDivision != 0'
                            break
                            
            elif tag.name == 'Tuplet':
                tupNum = tag.select_one('Number > text').text
                if tupNum != '3' and tupNum != '5' and tupNum != '7' and tupNum != '9':
#                     print 'Continue at measure #' + measure['number'] + ', which is with a tuplet not of 3, 5, 7 or 9'
                    break
                tupletID = tag['id']
                normalNotes = Decimal(tag.find('normalNotes').text)
                actualNotes = Decimal(tag.find('actualNotes').text)
                tupletRatio = normalNotes / actualNotes
            else: #tag.name == 'tick'
                ticking = int(tag.text)
        else:
            if track == 'multi_track' and time_acc != time['measure']:
#                 print 'Continue at measure #' + measure['number'] + ', the duration of last track is shorter than the measure'
                continue
                
            #Log if sum of note length doesn't match Time Signature 
            if abs(track_0_time_acc - time['measure']) >= 0.000000001:
                with open('C:/Users/BigData/Desktop/GuitarStaffBeatsError.log', 'a') as f:
                    f.write('[' + filename + ']\n' + 'time_acc of measure #' + measure['number'] + ' == ' + str(track_0_time_acc) + ', != 1\n')
                    
#             print 'Measure #' + measure['number'],
            thisMeasure = {track:thisMeasure[track][:-1] for track in thisMeasure}
            thisMeasure.update({'length':track_0_time_acc, 'filename':filename[:-5]})
            print {track:thisMeasure}

In [49]:
main('C:/Users/BigData/Desktop/mscx/')

7 Things.mscx
[=============== Staff #4 ===============]
{'single_track': {'track0': '0.125,1;0.125,1,0;0.1875,1,0;0.0625,1,0;0.0625,1,-1;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0', 'length': Decimal('1.0000'), 'filename': '7 Things'}}
{'single_track': {'track0': '0.125,1;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,-1;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0', 'length': Decimal('1.0000'), 'filename': '7 Things'}}
{'single_track': {'track0': '0.125,1;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.125,1,-1;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0', 'length': Decimal('1.0000'), 'filename': '7 Things'}}
{'single_track': {'track0': '0.125,1;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.125,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0;0.0625,1,0', 'length': Decimal('1.0000'), 'filename': '7 Things'}}
{'single_t